### Imports

In [31]:
import os
from pylab import *

from os.path import basename, dirname, isfile, isdir, join
from glob import glob
import numpy as np
import nibabel as nib
from tqdm import tqdm_notebook as tqdm
import matplotlib.animation as animation

### Load sample image

In [2]:
# ### Load image
# filepath = "/Volumes/hd_4tb/raw/connhc/000/conn030/gonogo/normalized.nii.gz"
# image    = nib.as_closest_canonical(nib.load(filepath))
# data     = image.get_data()

### Helper function

fps.  Frames per second

In [3]:
dpi = 100

def get_data_slice(data, axis):
    axis = axis.lower()
    assert axis in {"x", "y", "z"}, "invalid axis %s" % axis
    if axis == "x":
        L = data.shape[0] // 2
        return data[L, :, :]
    elif axis == "y":
        L = data.shape[1] // 2
        return data[:, L, :]
    elif axis == "z":
        L = data.shape[2] // 2
        return data[:, :, L]

def animate_nii(files, save_path, axis="z", num_dummies=3, TR=2, speed=1, norm=True):
    animate_frames(files, save_path, axis=axis, fps = (1 / TR) * speed)

def _create_plt_figure():
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    return fig, ax
    
def _load_frame(files, n, axis=None):
    data = np.load(files[n])
    frame = get_data_slice(data, axis)
    return np.ma.masked_where(frame == 0, frame)

def _flash(filepath):
    if filepath.endswith("s.npy"):
        return False
    return True

def at_most_level(num, filepath, low, high):
    name = basename(filepath)
    t = int(name.split(".")[0].split("_")[0])                                                                                                                                                                                                                                                                                                                           
    if t < low or t > high:
        return False
    if "_" not in name:
        return 0 <= num 
    return len(name.split("_")[1].replace("s.npy", "")) <= num

def animate_frames(files, save_path, axis="z", fps = 30, speed = 1):
    fps = fps * speed
    N = len(files)
    fig, ax = _create_plt_figure()
    
    cmap = plt.cm.RdBu_r
    cmap.set_bad("yellow")
    
    frame = _load_frame(files, 0, axis=axis)
    im = ax.imshow(frame, cmap=cmap)

    fig.set_size_inches([5,5])

    tight_layout()

    def update_img(n):
        frame = _load_frame(files, n, axis=axis)
        im.set_data(frame)
        if _flash(files[n]):
            cmap.set_bad("yellow")
        else:
            cmap.set_bad("black")
        im.set_cmap(cmap)
        return im

    ani = animation.FuncAnimation(fig, update_img, N, interval=fps)
    writer = animation.writers["ffmpeg"](fps=fps)

    ani.save(save_path, writer=writer, dpi=dpi)
    plt.close()

### Main

In [40]:
[f for f in files if at_most_level(4, f, 100, 135)]

['/Users/pstetz/Desktop/confidential/.project/interpolate/mse/processed_volumes_290/100.npy',
 '/Users/pstetz/Desktop/confidential/.project/interpolate/mse/processed_volumes_290/100_zaaas.npy',
 '/Users/pstetz/Desktop/confidential/.project/interpolate/mse/processed_volumes_290/100_zaas.npy',
 '/Users/pstetz/Desktop/confidential/.project/interpolate/mse/processed_volumes_290/100_zaazs.npy',
 '/Users/pstetz/Desktop/confidential/.project/interpolate/mse/processed_volumes_290/100_zas.npy',
 '/Users/pstetz/Desktop/confidential/.project/interpolate/mse/processed_volumes_290/100_zazas.npy',
 '/Users/pstetz/Desktop/confidential/.project/interpolate/mse/processed_volumes_290/100_zazs.npy',
 '/Users/pstetz/Desktop/confidential/.project/interpolate/mse/processed_volumes_290/100_zazzs.npy',
 '/Users/pstetz/Desktop/confidential/.project/interpolate/mse/processed_volumes_290/100_zs.npy',
 '/Users/pstetz/Desktop/confidential/.project/interpolate/mse/processed_volumes_290/100_zzaas.npy',
 '/Users/pste

In [42]:
TR = .25
directory = "volumes_290"
files = glob("/Users/pstetz/Desktop/confidential/.project/interpolate/mse/processed_%s/*" % directory)
files = [f for f in files if at_most_level(3, f, 100, 135)]
files = list(sorted(files))
save_path = "/Users/pstetz/Desktop/axis_%s.mp4"
for axis in ("x", "y", "z"):
    animate_nii(files, save_path % axis, axis = axis, TR=TR, speed=1)

### Scale guesses

In [38]:
from os.path import basename, join, isfile

directory = "volumes_290"
input_dir = "/Users/pstetz/Desktop/confidential/.project/interpolate/mse/%s/" % directory
output_dir = "/Users/pstetz/Desktop/confidential/.project/interpolate/mse/processed_%s/" % directory
mask_file = "/Users/pstetz/Desktop/confidential/.project/interpolate/MNI152_T1_2mm_brain_mask.nii.gz"
mask_data = nib.load(mask_file).get_fdata()

for filepath in tqdm(glob(join(input_dir, "*"))):
    name = basename(filepath)
    dst = join(output_dir, name)
    if isfile(dst):
        continue
    data = np.load(filepath)
    data = np.multiply(data, mask_data)
    _mean = np.mean(data)
    _std = np.std(data)
    data[:, :, :] = np.add(np.divide(np.subtract(data[:, :, :], _mean), _std), _mean)
    data = np.multiply(data, mask_data)
    np.save(dst, data)

/Users/pstetz/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


### cmap colors

In [5]:
colors = [
            'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
            'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
            'hot', 'afmhot', 'gist_heat', 'copper', 'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
            'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic',
            'twilight', 'twilight_shifted', 'hsv',
            'Pastel1', 'Pastel2', 'Paired', 'Accent',
            'Dark2', 'Set1', 'Set2', 'Set3',
            'tab10', 'tab20', 'tab20b', 'tab20c'
]